### Word 2 Vector

### Unit Test word 2 vec corpus construciton before we apply to all the data

In [47]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from gensim.models.word2vec import Word2Vec

In [2]:
data=pd.read_csv('yelp.csv')
data.tail()

,business_id,date,review_id,stars,text,type,user_id,cool,useful,funny
9995,VY_tvNUCCXGXQeSvJl757Q,2012-07-28,Ubyfp2RSDYW0g7Mbr8N3iA,3,First visit...Had lunch here today - used my G...,review,_eqQoPtQ3e3UxLE4faT6ow,1,2,0
9996,EKzMHI1tip8rC1-ZAy64yg,2012-01-18,2XyIOQKbVFb6uXQdJ0RzlQ,4,Should be called house of deliciousness!\n\nI ...,review,ROru4uk5SaYc3rg8IU7SQw,0,0,0
9997,53YGfwmbW73JhFiemNeyzQ,2010-11-16,jyznYkIbpqVmlsZxSDSypA,4,I recently visited Olive and Ivy for business ...,review,gGbN1aKQHMgfQZkqlsuwzg,0,0,0
9998,9SKdOoDHcFoxK5ZtsgHJoA,2012-12-02,5UKq9WQE1qQbJ0DJbc-B6Q,2,My nephew just moved to Scottsdale recently so...,review,0lyVoNazXa20WzUyZPLaQQ,0,0,0
9999,pF7uRzygyZsltbmVpjIyvw,2010-10-16,vWSmOhg2ID1MNZHaWapGbA,5,4-5 locations.. all 4.5 star average.. I think...,review,KSBFytcdjPKZgXKQnYQdkA,0,0,0


In [ ]:
#creating a sample list to unit test the preprocessed_reviews code below
sample_10=BOW.head(2)
sample_10['text'][0]

In [ ]:
# tqdm is for printing the status bar
from tqdm import tqdm
preprocessed_reviews = []
for sentence in tqdm(sample_10['text'].values):
    sentence=clean_review_text(sentence)
   # sentence=' '.join(sentence)
   # preprocessed_reviews.append(sentence.strip())
    preprocessed_reviews.append(sentence)
print(preprocessed_reviews)

In [11]:
import string
import nltk
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
def clean_review_text(review):
    """
    1. Remove Punctuation
    2. Remove Stop Words
    3. Apply SnowBall Stemmer to remove morphological affixes from words, leaving only the word stem.
    """
    stemmer = SnowballStemmer("english")
    no_punc=[c for c in review if c not in string.punctuation]
    no_punc=''.join(no_punc)
    return [stemmer.stem(word) for word in no_punc.split() if word.lower() not in stopwords.words('english')]
    

### Perform Word2Vec on whole data and build your own corpus

In [ ]:
# Train your own Word2Vec model using your own text corpus for whole data
# tqdm is for printing the status bar
from tqdm import tqdm
preprocessed_reviews = []
for sentence in tqdm(BOW['text'].values):
    sentence=clean_review_text(sentence)
    preprocessed_reviews.append(sentence)

In [3]:
### Precict the simillarity of wrods with the model we have built

In [ ]:
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
# min_count = 2 considers only words that occured atleast 2 times
w2v_model=Word2Vec(preprocessed_reviews,min_count=2,size=50)
print(w2v_model.wv.most_similar('great'))
print('='*50)
print(w2v_model.wv.most_similar('worst'))
print(w2v_model.wv['worst'])

In [4]:
### create list of all the words/Corpus form the model 

In [ ]:
w2v_words = list(w2v_model.wv.vocab)
print("number of words that occured minimum 2 times ",len(w2v_words))
print("sample words ", w2v_words[0:10])

In [ ]:
### Convert text into Vector using Average W2Vec  for whole sentence

In [ ]:
# average Word2Vec
# compute average word2vec for each rating.
sent_vectors = []; # the avg-w2v for each sentence/review is stored in this list
for sent in tqdm(preprocessed_reviews): # for each review/sentence
    sent_vec = np.zeros(50) # as word vectors are of zero length 50, you might need to change this to 300 if you use google's w2v
    cnt_words =0; # num of words with a valid vector in the sentence/review
    for word in sent: # for each word in a review/sentence
        if word in w2v_words:
            vec = w2v_model.wv[word]
            sent_vec += vec
            cnt_words += 1
    if cnt_words != 0:
        sent_vec /= cnt_words
    sent_vectors.append(sent_vec)
print(len(sent_vectors))
print(len(sent_vectors[0]))

In [19]:
#Source: http://nadbordrozd.github.io/blog/2016/05/20/text-classification-with-word2vec/ 
from tqdm import tqdm
class MeanEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        # if a text is empty we should return a vector of zeros
        # with the same dimensionality as all the other vectors
        self.dim = len(word2vec[0])

    def fit(self, X, y):
        return self

    def transform(self, X):
        sent_vectors = []; # the avg-w2v for each sentence/review is stored in this list
        for sent in tqdm(X): # for each review/sentence
            sent_vec = np.zeros(50) # as word vectors are of zero length 50, you might need to change this to 300 if you use google's w2v
            cnt_words =0; # num of words with a valid vector in the sentence/review
            for word in sent: # for each word in a review/sentence
                if word in w2v_words:
                    vec = w2v_model.wv[word]
                    sent_vec += vec
                    cnt_words += 1
            if cnt_words != 0:
                sent_vec /= cnt_words
            sent_vectors.append(sent_vec)
        return sent_vectors

In [ ]:
pipeline_W2V=Pipeline([('MeanTransformer',MeanEmbeddingVectorizer(w2v_model)),
#                       ('tfidf',TfidfTransformer()),
                       ('algorithm',RandomForestClassifier())])


In [9]:
df=data[(data['stars']==5 )| (data['stars']==1)]
X_W2V= df['text']
y_W2V=df['stars']

In [12]:
#Take input text and clean punctuations , stop words etc with custom analyzer
cvo=CountVectorizer(analyzer=clean_review_text)
#Replace X with the cleaned text
X_W2V=cvo.fit_transform(X_W2V)

In [20]:
sent_vectors=[[]]
X_train_W2V, X_test_W2V, y_train_W2V, y_test_W2V = train_test_split(X_W2V,y_W2V, test_size=0.3, random_state=101)
model=MeanEmbeddingVectorizer(X_train_W2V)
sent_vectors=model.transform(X_train_W2V)
print(sent_vectors[0])

0it [00:00, ?it/s]


NameError: name 'w2v_words' is not defined

In [ ]:
#X_train_W2V, X_test_W2V, y_train_W2V, y_test_W2V = train_test_split(X_W2V,y_W2V, test_size=0.3, random_state=101)
#scores.append(accuracy_score(model.fit(X_train, y_train).predict(X_test), y_test))
pipeline_W2V.fit(X_W2V, y_W2V)
pipeline_W2V.transform(X_W2V, y_W2V)
predictions_W2V=pipeline_W2V.predict("BOW[0]")
print(predictions_W2V)

In [ ]:
predictions_W2V=pipeline_W2V.predict("U can go there n check the car out. If u wanna buy 1 there? That's wrong move! If u even want a car service from there? U made a biggest mistake of ur life!! I had 1 time asked my girlfriend to take my car there for an oil service, guess what? They ripped my girlfriend off by lying how bad my car is now. If without fixing the problem. Might bring some serious accident. Then she did what they said. 4 brand new tires, timing belt, 4 new brake pads. U know why's the worst? All of those above I had just changed 2 months before!!! What a trashy dealer is that? People, better off go somewhere!")
print(predictions_W2V)

In [ ]:
predictions_W2V[0:100]

In [ ]:
from scipy.spatial.distance import euclidean
# first distance should be much smaller than second
euclidean(w2v_model['wife'], w2v_model['girlfriend']), euclidean(w2v_model['wife'], w2v_model['problem'])

In [ ]:
euclidean(w2v_model['problem'], w2v_model['good'])

In [ ]:
mp=MeanEmbeddingVectorizer(BOW[0])
mp.transform()
        

In [ ]:
mean_vec_tr = MeanEmbeddingVectorizer(w2v_model)
doc_vec = mean_vec_tr.transform(w2v_words)

In [ ]:
#Source: http://nadbordrozd.github.io/blog/2016/05/20/text-classification-with-word2vec/ 
class TfidfEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        self.word2weight = None
        self.dim = len(word2vec.itervalues().next())

    def fit(self, X, y):
        tfidf = TfidfVectorizer(analyzer=lambda x: x)
        tfidf.fit(X)
        # if a word was never seen - it must be at least as infrequent
        # as any of the known words - so the default idf is the max of 
        # known idf's
        max_idf = max(tfidf.idf_)
        self.word2weight = defaultdict(
            lambda: max_idf,
            [(w, tfidf.idf_[i]) for w, i in tfidf.vocabulary_.items()])

        return self

    def transform(self, X):
        return np.array([
                np.mean([self.word2vec[w] * self.word2weight[w]
                         for w in words if w in self.word2vec] or
                        [np.zeros(self.dim)], axis=0)
                for words in X
            ])

In [22]:
import pandas as pd
 
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
 
# this is a very toy example, do not try this at home unless you want to understand the usage differences
docs=["the house had a tiny little mouse",
      "the cat saw the mouse",
      "the mouse ran away from the house",
      "the cat finally ate the mouse",
      "the end of the mouse story"
     ]

In [23]:
#instantiate CountVectorizer()
cv=CountVectorizer()
 
# this steps generates word counts for the words in your docs
word_count_vector=cv.fit_transform(docs)
 

In [37]:
cv.get_feature_names()

['ate',
 'away',
 'cat',
 'end',
 'finally',
 'from',
 'had',
 'house',
 'little',
 'mouse',
 'of',
 'ran',
 'saw',
 'story',
 'the',
 'tiny']

In [31]:
word_count_vector[:].toarray()

array([[0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 2, 0],
       [0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 2, 0],
       [1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 2, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 2, 0]])

In [32]:
tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)

In [33]:
tfidf_transformer.fit(word_count_vector) 

TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)

In [36]:
tfidf_transformer.idf_

array([2.09861229, 2.09861229, 1.69314718, 2.09861229, 2.09861229,
       2.09861229, 2.09861229, 1.69314718, 2.09861229, 1.        ,
       2.09861229, 2.09861229, 2.09861229, 2.09861229, 1.        ,
       2.09861229])

In [38]:
count_vector=cv.transform(docs)

In [42]:
count_vector[:].toarray()

array([[0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 2, 0],
       [0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 2, 0],
       [1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 2, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 2, 0]])

In [43]:
tf_idf_vector=tfidf_transformer.transform(count_vector)

In [45]:
tf_idf_vector[:].toarray()

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.49356209, 0.39820278, 0.49356209, 0.23518498,
        0.        , 0.        , 0.        , 0.        , 0.23518498,
        0.49356209],
       [0.        , 0.        , 0.48334378, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.28547062,
        0.        , 0.        , 0.59909216, 0.        , 0.57094124,
        0.        ],
       [0.        , 0.45709287, 0.        , 0.        , 0.        ,
        0.45709287, 0.        , 0.36877965, 0.        , 0.2178072 ,
        0.        , 0.45709287, 0.        , 0.        , 0.43561441,
        0.        ],
       [0.51392301, 0.        , 0.41462985, 0.        , 0.51392301,
        0.        , 0.        , 0.        , 0.        , 0.24488707,
        0.        , 0.        , 0.        , 0.        , 0.48977413,
        0.        ],
       [0.        , 0.        , 0.        , 0.49175319, 0.        ,
        0.        , 0.        , 

In [51]:
from sklearn.feature_extraction.text import TfidfVectorizer 
S = ['wife',
 'took',
 'birthday',
 'breakfast',
 'excel',
 'weather',
 'perfect',
 'made',
 'sit',
 'outsid',
 'overlook']
model = TfidfVectorizer()
model.fit(S)
# we are converting a dictionary with word as a key, and the idf as a value
dictionary = dict(zip(model.get_feature_names(), list(model.idf_)))

In [52]:
dictionary

{'birthday': 2.791759469228055,
 'breakfast': 2.791759469228055,
 'excel': 2.791759469228055,
 'made': 2.791759469228055,
 'outsid': 2.791759469228055,
 'overlook': 2.791759469228055,
 'perfect': 2.791759469228055,
 'sit': 2.791759469228055,
 'took': 2.791759469228055,
 'weather': 2.791759469228055,
 'wife': 2.791759469228055}

In [68]:
s1=[["ant","bed"],["cat","den"]]
' '.join(s1[ele]

In [ ]:
lst_str=

In [74]:
list(map(' '.join,s1))

['ant bed', 'cat den']

In [72]:
for ele in s1  lambda el: 
    new=[' '.join(s1[ele])
    
print(new)

TypeError: list indices must be integers or slices, not list

In [61]:
s2=''

'ab'